<a href="https://colab.research.google.com/github/Daeijavad/Deep-Learning/blob/main/HW6_LSTMSentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Recurrent Neural Networks

Welcome to HomeWork 6  &#128522;&#9996;

## Part II : Sentiment Analysis Using RNNs

In this part, you will use reccurent neural networks to solve the Sentiment Analysis problem. 

At first, we load the required packages. Add other packages if you need them.

In [ ]:
import tensorflow as tf
# or other libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

Then, we load the dataset:

In [ ]:
sentiment_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/dataset.txt', sep='\t')
sentiment_data.columns =['Class', 'Data']

Here, you can see some of the data:

In [ ]:
sentiment_data.head()

,Class,Data
0,1,this was the first clive cussler i've ever rea...
1,1,i liked the Da Vinci Code a lot.
2,1,i liked the Da Vinci Code a lot.
3,1,I liked the Da Vinci Code but it ultimatly did...
4,1,that's not even an exaggeration ) and at midni...


From now implement your model.
<br>
Please add cells and explain yours developing steps and your results.

# Pre proccessing

> for this part we first make a martix corresponding to data according to the count of each word

> then we reshape it for importing to LSTM

> at end we split them into train and test (test contains 10% of data)

using https://machinelearningmastery.com/prepare-text-data-deep-learning-keras/

In [ ]:
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
import numpy as np

tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentiment_data['Data'])
dataX = tokenizer.texts_to_matrix(sentiment_data['Data'], mode='count')

dataX = np.reshape(dataX, (dataX.shape[0], dataX.shape[1], 1))

X_train, X_test, y_train, y_test = train_test_split(dataX, sentiment_data['Class'],shuffle = True, test_size=0.1, random_state=42)

In [ ]:
#sample data
print(X_train[1])

[[0.]
 [1.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]


# model architecture and train

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

model = Sequential()
model.add(LSTM(256, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 256)               264192    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 264,449
Trainable params: 264,449
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train, y_train, validation_data = (X_test,y_test), epochs=3, batch_size=64)




Train on 6225 samples, validate on 692 samples
Epoch 1/3





6225/6225 [==============================] - 394s 63ms/step - loss: 0.6850 - acc: 0.5653 - val_loss: 0.6817 - val_acc: 0.5997
Epoch 2/3
6225/6225 [==============================] - 376s 60ms/step - loss: 0.6853 - acc: 0.5667 - val_loss: 0.6765 - val_acc: 0.5997
Epoch 3/3
6225/6225 [==============================] - 371s 60ms/step - loss: 0.6846 - acc: 0.5667 - val_loss: 0.6776 - val_acc: 0.5997


> for having a better performance it's good to use word embedding

# Better Model Using Word Embedding

In [ ]:
from keras.preprocessing import sequence

tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentiment_data['Data'])
sequences = tokenizer.texts_to_sequences(sentiment_data['Data'])
sequences_same_size = sequence.pad_sequences(sequences)

X_train, X_test, y_train, y_test = train_test_split(sequences_same_size, sentiment_data['Class'], shuffle = True, test_size=0.1, random_state=42)

> this is list that contains words with the number assinged to each word

In [ ]:
print(tokenizer.word_counts)

OrderedDict([('this', 214), ('was', 1175), ('the', 3224), ('first', 104), ('clive', 1), ('cussler', 1), ("i've", 11), ('ever', 96), ('read', 113), ('but', 295), ('even', 27), ('books', 29), ('like', 974), ('relic', 1), ('and', 2153), ('da', 1999), ('vinci', 2000), ('code', 1998), ('were', 96), ('more', 103), ('plausible', 1), ('than', 14), ('i', 4645), ('liked', 101), ('a', 1305), ('lot', 22), ('it', 720), ('ultimatly', 1), ("didn't", 8), ('seem', 1), ('to', 808), ('hold', 2), ("it's", 180), ('own', 4), ("that's", 6), ('not', 198), ('an', 225), ('exaggeration', 1), ('at', 110), ('midnight', 1), ('we', 219), ('went', 115), ('wal', 1), ('mart', 1), ('buy', 2), ('which', 252), ('is', 1518), ('amazing', 16), ('of', 424), ('course', 7), ('loved', 256), ('now', 21), ('want', 254), ('something', 7), ('better', 96), ('different', 4), ('thought', 102), ('great', 95), ('same', 87), ('with', 288), ('kite', 1), ('runner', 1), ('actually', 14), ('good', 127), ('movie', 783), ('pretty', 32), ('book'

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Embedding

model = Sequential()
model.add(Embedding(len(tokenizer.word_counts), 100, input_length=sequences_same_size.shape[1]))
model.add(LSTM(256))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 933, 100)          222100    
_________________________________________________________________
lstm_3 (LSTM)                (None, 256)               365568    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 587,925
Trainable params: 587,925
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(X_train, y_train, validation_data = (X_test,y_test), epochs=3, batch_size=64)

Train on 6225 samples, validate on 692 samples
Epoch 1/3
6225/6225 [==============================] - 181s 29ms/step - loss: 0.3010 - acc: 0.8859 - val_loss: 0.0574 - val_acc: 0.9798
Epoch 2/3
6225/6225 [==============================] - 181s 29ms/step - loss: 0.0268 - acc: 0.9920 - val_loss: 0.0450 - val_acc: 0.9899
Epoch 3/3
6225/6225 [==============================] - 181s 29ms/step - loss: 0.0103 - acc: 0.9974 - val_loss: 0.0223 - val_acc: 0.9928
